# 1. 處理 train data
train/test來源 ：https://github.com/wshuyi/demo-chinese-text-classification-lstm-keras/blob/master/demo.ipynb

code 來源 ：https://blog.csdn.net/Jerryzhangjy/article/details/110209984

##import

In [1]:
from google.colab import drive
drive.mount("/content/drive")
path =  "/content/drive/Shareddrives/AI_project/"

Mounted at /content/drive


In [2]:
!pip install keras.utils
! pip3 install transformers
! pip3 install keras
! pip3 install tensorflow


  Created wheel for keras.utils: filename=keras_utils-1.0.13-cp37-none-any.whl size=2657 sha256=5ca2a124739088ba4b4067a883dd70616504e573f62f94ac0905b0351395112d
  Stored in directory: /root/.cache/pip/wheels/46/25/27/7707005c1cb27e1ffc8277b004ac295e34767b02b44d73d6be
Successfully built keras.utils
     |████████████████████████████████| 2.5MB 5.0MB/s 
     |████████████████████████████████| 3.3MB 31.5MB/s 
     |████████████████████████████████| 901kB 37.7MB/s 


In [3]:
import pandas as pd
import numpy as np


In [4]:
import torch
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


##讀檔

In [8]:
from pathlib import Path
#mypath = Path("demo-chinese-text-classification-lstm-keras")
#-------------Modify the train_path----------------

train_path = path + 'data/train/dianping.csv'

#-----------------------------------
df = pd.read_csv(train_path)
df.head(3)

,comment,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。 我感觉口味确实很一般很一般。上菜相当快，我敢说...,0
1,菜品丰富质量好，服务也不错！很喜欢！,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,0


#XLnet

In [9]:
T_sentences = df.comment.values # T means Total
T_labels = np.array(df.sentiment.values,dtype=np.int32)

In [ ]:
#!pip install knockknock==0.1.7
#!pip install sentencepiece
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
!unzip cased_L-12_H-768_A-12.zip

In [ ]:
!pip install keras_xlnet
import os
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint #, ATTENTION_TYPE_BI, ATTENTION_TYPE_UNI
from transformers import XLNetTokenizer, XLNetLMHeadModel, XLNetModel


In [12]:
import torch

checkpoint_path = 'xlnet_cased_L-12_H-768_A-12' 
MEMLEN=512
BATCH_SIZE=16

#tokenizer = XLNetTokenizer.from_pretrained("hfl/chinese-xlnet-base")
#model = XLNetModel.from_pretrained('hfl/chinese-xlnet-base')


## training data

In [13]:
LS = len(T_sentences)
training_samples = (int)(LS * 0.9)
sentences = T_sentences #[:training_samples]
labels = T_labels #[:training_samples]

#test_sentences = T_sentences[ training_samples : LS ]
#test_labels = T_labels[training_samples : LS]


In [16]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
from keras.preprocessing.sequence import pad_sequences
tokenizer = XLNetTokenizer.from_pretrained("hfl/chinese-xlnet-base")
'''
input_ids = []
for sent in sentences:
    encoded_sent = torch.tensor(tokenizer.encode(sent,add_special_tokens = True))
    input_ids.append(encoded_sent)
'''
MAX_LEN = 200
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


# Create attention masks
attention_masks = []
# For each sentence...
for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)



In [17]:
# Use train_test_split to split our data into train and validation sets for
# training
from sklearn.model_selection import train_test_split
from transformers import AdamW, BertConfig,XLNetForSequenceClassification
# Use 90% for training and 10% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                             random_state=2018, test_size=0.1)

In [18]:
# Convert all inputs and labels into torch tensors, the required datatype 
# for our model.
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [19]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# For fine-tuning BERT on a specific task, the authors recommend a batch size of # 16 or 32.
batch_size = 16
# Create the DataLoader for our training set.
print(train_inputs.size(),train_masks.size(),train_labels.size())
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# Create the DataLoader for our validation set.
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

torch.Size([1800, 200]) torch.Size([1800, 200]) torch.Size([1800])


In [20]:
#from transformers import BertForSequenceClassification, AdamW, BertConfig
# Parameters
from transformers import XLNetConfig
seed = 42
use_fp16 = False
bs = 16
model_type = 'xlnet'
pretrained_model_name = 'hfl/chinese-xlnet-base'
model_class, tokenizer_class, config_class = (XLNetForSequenceClassification, XLNetTokenizer,
XLNetConfig)

model = XLNetForSequenceClassification.from_pretrained('hfl/chinese-xlnet-base',num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False)

Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['sequence_summary.summary.bias', 'logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [21]:
# Get all of the model's parameters as a list of tuples.

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters,
                     lr=2e-5)

In [22]:
from transformers import get_linear_schedule_with_warmup
'''
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
'''

epochs = 3 #
total_steps = len(train_dataloader) * epochs
print(total_steps)
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

452


In [23]:
import numpy as np
import time
import datetime
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
import random
import numpy as np

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_loss = 0
    model.train() 
    #----------------------- For each batch of training data -----------------------
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)           
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # As we unpack the batch, we'll also copy each tensor to the GPU using the  `to` method.
        b_input_ids = batch[0].long().to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].long().to(device)
        # Always clear any previously calculated gradients before performing a backward pass
        # backward pass. PyTorch doesn't do this automatically because accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
          # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        total_loss += (loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()  # Update the learning rate.
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch) # Add batch to GPU
        b_input_ids, b_input_mask, b_labels = batch # Unpack the inputs from our dataloader
        
        # Tell the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            # token_type_ids is the same as the "segment ids", which differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy
        # Track the number of batches
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:28:22.
  Batch    80  of    113.    Elapsed: 0:57:00.

  Average training loss: 0.45
  Training epcoh took: 1:20:17

Running Validation...
  Accuracy: 0.92
  Validation took: 0:02:56

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:28:12.
  Batch    80  of    113.    Elapsed: 0:56:35.

  Average training loss: 0.23
  Training epcoh took: 1:19:34

Running Validation...
  Accuracy: 0.91
  Validation took: 0:02:50

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:27:40.
  Batch    80  of    113.    Elapsed: 0:55:28.

  Average training loss: 0.16
  Training epcoh took: 1:17:55

Running Validation...
  Accuracy: 0.92
  Validation took: 0:02:50

======== Epoch 4 / 4 ========
Training...
  Batch    40  of    113.    Elapsed: 0:27:55.
  Batch    80  of    113.    Elapsed: 0:55:36.

  Average training loss: 0.12
  Training epcoh took: 1:18:00

Ru

In [ ]:
import plotly.express as px
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

## Tesing dataset

In [25]:
df_test = pd.read_csv(path + 'data/1_column_title/simple_label_2kind.csv')
test_sentences = df_test['title']
test_labels = df_test['label']

In [26]:
import pandas as pd
# Load the dataset into a pandas dataframe.
'''
# Create sentence and label lists
sentences = df.sentence.values
labels = np.array(df.label.values,dtype=np.int32)
'''

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
# For every sentence...
for sent in test_sentences:
    # `encode` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    encoded_sent = torch.tensor(tokenizer.encode(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                   ))
    
    input_ids.append(encoded_sent)
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                          dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask) 
# Convert to tensors.
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_labels)
# Set the batch size.  
batch_size = 16  
# Create the DataLoader.
prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [27]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
  logits = outputs[0]
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
print('DONE.')
print('Positive samples: %d of %d (%.2f%%)' % (test_labels.sum(), len(test_labels), (test_labels.sum() / len(test_labels) * 100.0)))

Predicting labels for 111 test sentences...
DONE.
Positive samples: 42 of 111 (37.84%)


In [28]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []
# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')
# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)
  # Combine the predictions for each batch into a single list of 0s and 1s.
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = [item for sublist in true_labels for item in sublist]
# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)
print('MCC: %.3f' % mcc)

Calculating Matthews Corr. Coef. for each batch...
MCC: 0.425


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [29]:
count = 0
for i in range(len(flat_true_labels)):
  if int(flat_predictions[i]) ==  int(flat_true_labels[i]):
      count +=1
  else:
    None
    #print("title : ",test_sentences[i], flat_true_labels[i]," wrong label ",flat_predictions[i])
print(count, "正确率： " ,count/len(flat_true_labels))

80 正确率：  0.7207207207207207


#利用原始Model_output

In [ ]:
#!pip install snownlp
from snownlp import SnowNLP

In [ ]:
media = "chinatimes"
media_names = ["chinatimes","cna","cts","ebc","ettoday"]
catego = '政治'
sample_cnt = 200
'''
udn, storm
sector.size :
  chinatimes     9129
  cna            4060
  cts            1052
  ebc            1414
  ettoday        8469
  ltn            8518
  media             1
  setn           5452
  storm          1281 : ['公共政策', '國內'] 
  udn           11279 : 社會
'''

In [ ]:
path =  "/content/drive/Shareddrives/AI_project/"
df = pd.read_csv(path + 'data/news_all.csv')
df_3col = df[['media', 'category', 'title']]
sector = df_3col.groupby('media')
sector.size()


for media in median_names:
  m = sector.get_group(media) #m=media
  mc = m.groupby('category') #mc=media category
  mc_P = mc.get_group("['公共政策', '國內']")['title'] # mc_P's P means politic
  data_array = []
  for line in mc_P[:sample_cnt] : 
    simple = SnowNLP(line).han #轉簡體
    data_array.append(simple)
  data = pd.DataFrame(data_array, columns = ['title'])
  #--------------------------------------------------------------

  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  # For every sentence...
  for sent in data_array:
      encoded_sent = tokenizer.encode(sent,add_special_tokens = True,)    
      input_ids.append(encoded_sent)
  # Pad our input tokens
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                            dtype="long", truncating="post", padding="post")
  # Create attention masks
  attention_masks = []
  # Create a mask of 1s for each token followed by 0s for padding
  for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask) 
  # Convert to tensors.
  prediction_inputs = torch.tensor(input_ids)
  prediction_masks = torch.tensor(attention_masks)
  prediction_labels = torch.tensor(test_labels)
  # Set the batch size.  
  batch_size = 32  
  # Create the DataLoader.
  prediction_data = TensorDataset(prediction_inputs, prediction_masks)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  #--------------------------------------------------------------
  # Prediction on test set
  print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  predictions = []
  # Predict 
  for batch in prediction_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask = batch
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    '''label_ids = b_labels.to('cpu').numpy()'''
    
    # Store predictions and true labels
    predictions.append(logits)
    '''true_labels.append(label_ids)'''
  print('DONE.')
  #--------------------------------------------------------------
  matthews_set = []
  print('Calculating Matthews Corr. Coef. for each batch...')
  # For each input batch...
  for i in range(len(predictions)):
    
    #Pick the label with the highest value and turn this in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
    
    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(pred_labels_i, pred_labels_i)                
    matthews_set.append(matthews)

  flat_predictions = [item for sublist in predictions for item in sublist]
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
  #--------------------------------------------------------------
  pos_idx = []
  neg_idx  = [] 
  pos_cnt = 0
  neg_cnt = 0

  for i in range(len(flat_predictions)):
    predict = flat_predictions[i]
    if (predict==0) : 
      neg_cnt+=1
      neg_idx.append(i)  
    else:
      pos_cnt+=1
      pos_idx .append(i)
  #----------------------------------------------------------------
  path =  "/content/drive/Shareddrives/AI_project/"
  filepath = path+'result/'+ media +'/'+"XLnet_"

  path = filepath+'.txt'
  f = open(path, 'w')
  f.write(str(pos_cnt)+" ")
  f.close()
  f = open(path, 'a')
  f.write(str(neg_cnt))
  f.close()
  #----------------------------------------------------------------
  pos = []
  neg = [] 
  for idx in pos_idx : pos.append(data_array[idx])
  for idx in neg_idx : neg.append(data_array[idx])

  p = pd.DataFrame(pos)
  p.to_csv(filepath+"pos.csv")


  neg = pd.DataFrame(neg)
  neg.to_csv(filepath+"neg.csv")
  print(pos_cnt, neg_cnt)

# 不用的東西

In [ ]:
path =  "/content/drive/Shareddrives/AI_project/"
df = pd.read_csv(path + 'data/news_all.csv')
df_3col = df[['media', 'category', 'content']]
sector = df_3col.groupby('media')
#----------------------------------------------------------------
for media in ["中時","中央社", "三立"]:
  m = sector.get_group(media) #m=media
  mc = m.groupby('category') #mc=media category
  mc_P = mc.get_group('政治')['content'] # mc_P's P means politic
  data_array = []
  for line in mc_P : 
    simple = SnowNLP(line).han #轉簡體
    data_array.append(simple)
  data = pd.DataFrame(data_array, columns = ['text'])
  data['text'] = data.text.apply(lambda x: " ".join(jieba.cut(x)))
  #----------------------------------------------------------------
  tokenizer = Tokenizer(num_words=max_words)
  tokenizer.fit_on_texts(data.text)
  sequences = tokenizer.texts_to_sequences(data.text)
  type(sequences)
  data_cnt = (len(sequences))
  print("data_cnt : ",data_cnt)
  #----------------------------------------------------------------
  data = pad_sequences(sequences, maxlen=maxlen)
  #----------------------------------------------------------------
  pos_idx = []
  nu_idx  = []
  neg_idx  = [] 
  pos_cnt = 0
  nu_cnt = 0
  neg_cnt = 0

  for i in range(data_cnt):
    prediction = model.predict(data[i:i+1])
    predict_value = (prediction[0][0])
    if (predict_value)<0.3 : 
      neg_cnt+=1
      neg_idx.append(i)
    else:
      if predict_value>0.6:
        pos_cnt+=1
        pos_idx .append(i)
      else :
        nu_cnt+=1
        nu_idx .append(i)
  #----------------------------------------------------------------
  path =  "/content/drive/Shareddrives/AI_project/"
  filepath = path+'result/'+ media +'/'+"Demo_"
  print(pos_cnt, nu_cnt, neg_cnt)

  path = filepath+'.txt'
  f = open(path, 'w')
  f.write(str(pos_cnt)+" ")
  f.close()
  f = open(path, 'a')
  f.write(str(nu_cnt)+" ")
  f.write(str(neg_cnt))
  f.close()
  #----------------------------------------------------------------
  pos = []
  nu = []
  neg = [] 
  for idx in pos_idx : pos.append(data_array[idx])
  for idx in nu_idx : nu.append(data_array[idx])
  for idx in neg_idx : neg.append(data_array[idx])

  p = pd.DataFrame(pos)
  p.to_csv(filepath+"pos.csv")

  nu = pd.DataFrame(nu)
  nu.to_csv(filepath+"nu.csv")

  neg = pd.DataFrame(neg)
  neg.to_csv(filepath+"neg.csv")